# Analyze data

# 1. Imports

## 1.1 Packages

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

## 1.2 Options

## 1.3 Dataset

In [2]:
df_train = pd.read_csv("../data/01_raw/train.csv")
df_stores = pd.read_csv("../data/01_raw/stores.csv")
df_transactions = pd.read_csv("../data/01_raw/transactions.csv").sort_values(["store_nbr", "date"])
df_oil = pd.read_csv("../data/01_raw/oil.csv")
df_holidays = pd.read_csv("../data/01_raw/holidays_events.csv")

# 2. Prepare data

In [3]:
# Prepare datetime
df_train["date"] = pd.to_datetime(df_train.date)
df_transactions["date"] = pd.to_datetime(df_transactions.date)

In [4]:
df_train.sample(5)

,id,date,store_nbr,family,sales,onpromotion
658736,658736,2014-01-06,41,MAGAZINES,2.0,0
211074,211074,2013-04-29,31,CELEBRATION,0.0,0
1297671,1297671,2015-01-01,2,GROCERY I,0.0,0
1624261,1624261,2015-07-03,33,BABY CARE,0.0,0
2901373,2901373,2017-06-21,17,GROCERY II,7.0,0


In [5]:
df_transactions.sample(5)

,date,store_nbr,transactions
70891,2016-12-21,31,2178
24075,2014-06-04,26,641
8173,2013-06-27,36,784
71912,2017-01-11,44,4271
62771,2016-07-17,50,2528


In [6]:
df_stores.sample(5)

,store_nbr,city,state,type,cluster
34,35,Playas,Guayas,C,3
32,33,Quevedo,Los Rios,C,3
21,22,Puyo,Pastaza,C,7
5,6,Quito,Pichincha,D,13
46,47,Quito,Pichincha,A,14


In [7]:
df_oil.sample(5)

,date,dcoilwtico
343,2014-04-25,100.85
1070,2017-02-07,52.19
1050,2017-01-10,50.82
544,2015-02-02,49.25
376,2014-06-11,105.04


In [8]:
df_holidays.sample(5)

,date,type,locale,locale_name,description,transferred
183,2015-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
311,2017-05-14,Event,National,Ecuador,Dia de la Madre,False
174,2015-05-24,Holiday,National,Ecuador,Batalla de Pichincha,False
181,2015-07-23,Holiday,Local,Cayambe,Cantonizacion de Cayambe,False


In [9]:
df_holidays.type.value_counts(dropna=False)

type
Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: count, dtype: int64

In [17]:
df_holidays.locale.value_counts(dropna=False)

locale
National    174
Local       152
Regional     24
Name: count, dtype: int64

In [10]:
df_temp = df_train.merge(df_transactions, on=["date", "store_nbr"], how="left")
df_temp

,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,NaN
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,2155.0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2155.0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,2155.0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2155.0


In [11]:
df_oil["date"] = pd.to_datetime(df_oil.date)
# Resample
df_oil = df_oil.set_index("date").dcoilwtico.resample("D").sum().reset_index()
# Interpolate
df_oil["dcoilwtico"] = np.where(df_oil["dcoilwtico"] == 0, np.nan, df_oil["dcoilwtico"])
df_oil["dcoilwtico_interpolated"] = df_oil.dcoilwtico.interpolate()

# 3. Analyze data

## 3.1 Transactions

In [12]:
# px.line(df_transactions.sort_values(["store_nbr", "date"]), x='date', y='transactions', color='store_nbr',title = "Transactions" )

In [13]:
df_tp = df_transactions.copy()
df_tp["year"] = df_tp.date.dt.year
df_tp["month"] = df_tp.date.dt.month
fig = px.box(df_tp, x="year", y="transactions" , color = "month", title = "Transactions")
del df_tp

In [14]:
fig = px.scatter(df_temp, x="transactions", y="sales", trendline="ols", trendline_color_override="red")

In [15]:
p = df_oil.melt(id_vars=['date'] + list(df_oil.keys()[5:]), var_name='Legend')
fig = px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend', title="Daily Oil Price")
del p

In [16]:
df_sales = df_train.set_index("date").groupby("store_nbr").resample("D").sales.sum().reset_index()
fig = px.line(df_sales, x="date", y="sales", color="store_nbr", title="Daily total sales of the stores")
del df_sales